In [1]:
import sys, os, random
sys.path.insert(0,os.path.expanduser('~/git/pyzx')) # git version
sys.path.insert(0,'/workspaces/pyzx')
import pyzx as zx
from pyzx.pauliweb import preprocess, compute_pauli_webs

This notebook demonstrates using PyZX's simplifier to construct the reduced ZX diagram of a random Clifford+T circuit, then using PyZX's gflow-finding algorithm to automatically associate each node `n` in the diagram to an integer `order[n]` and a Pauli web (a.k.a. _correlation surface_) `web[n]` with the following properties:

1. the boundary of `web[n]` consists of `n` and inputs
2. `web[n]` connects to `n` by a single edge, which is the same colour as `n`
3. for non-Clifford nodes `m, n`, if `m` is in `web[n].vertices()` then `order[m] < order[n]`

The values of `web[n]` at non-Clifford nodes and at outputs should be enough data to deterministically implement the diagram via lattice surgery and either measure at the end or compute the updated Pauli frame.

In [7]:
# Generate a random CNOT, H, T circuit
random.seed(1330)
c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=3, depth=40)
for g in c.gates: print(g)
zx.draw(c)

CNOT(0,1)
CNOT(1,0)
CNOT(2,0)
HAD(2)
CNOT(2,1)
HAD(2)
CNOT(1,0)
CNOT(0,2)
CNOT(1,2)
T(0)
T(1)
HAD(2)
T(2)
HAD(1)
HAD(0)
CNOT(0,2)
CNOT(0,2)
CNOT(1,0)
T(2)
HAD(2)
HAD(1)
CNOT(1,0)
CNOT(0,1)
CNOT(2,1)
T(0)
CNOT(2,0)
T(0)
T(0)
T(0)
CNOT(2,0)
CNOT(0,2)
T(2)
T(1)
T(1)
T(0)
HAD(1)
T(1)
CNOT(1,0)
CNOT(1,2)
CNOT(1,0)


In [8]:
# Convert to a ZX diagram and call the full_reduce procedure on it (PyZX's main ZX diagram optimisation pass)
g = c.to_graph()
zx.full_reduce(g)

# Normalise compacts the circuit visually and ensures every input/output is connected to a Z spider
g.normalize()

# For simplicity, preprocess strips off any local Clifford unitaries and saves them in in_circ and out_circ,
# then introduces a dummy Z and X node at every output, so we can compute Z- and X-bounded Pauli webs there
in_circ, out_circ = preprocess(g)

# Compute the time-ordering on nodes (which is only important for the non-Clifford nodes) and compute the Pauli
# webs for every node.
order, webs = compute_pauli_webs(g)

# Draw the simplified ZX diagram. Note blue edges correspond to edges with Hadamard gates
zx.draw(g, labels=True)

In [9]:
# Once the Pauli webs have been computed, a specific web can be highlighted by `zx.draw` by passing it in as
# an optional argument. Note that webs change color when they cross Hadamard edges.
zx.draw(g, labels=True, pauli_web=webs[67])

We now show how this works in some simpler cases. The first is a single T gate.

The T gate becomes a single, 1-legged phase gadget, connected to the input. This can be implemented by Z-merging a T magic state, then doing either an X or a Y measurement, depending on the parity of the Pauli web.

In [5]:
c = zx.qasm("""
qreg q[1];
t q[0];
""")
zx.draw(c)

g = c.to_graph()
zx.full_reduce(g)
in_circ, out_circ = preprocess(g)
order, webs = compute_pauli_webs(g)
print(f'{in_circ.gates} {out_circ.gates}')

# highlight the web associated to the T spider
zx.draw(g, labels=True, pauli_web=webs[3])

[] []


The next example is a circuit with some CNOT gates and a T gate, which can be simplified to a single phase gadget. I'm doing this manually here, since the automated simplifier comes up with a different answer (which is equivalent to this one, up to local Cliffords, but less clear what is going on).

In [6]:
c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
cx q[1], q[2];
t q[2];
cx q[1], q[2];
cx q[0], q[1];
""")
zx.draw(c)

# manual ZX simplification to get a single phase gadget
g = c.to_graph()
zx.simplify.gadgetize(g, graphlike=False)
zx.basicrules.strong_comp(g, 5, 7)
zx.simplify.spider_simp(g, quiet=True)
zx.basicrules.strong_comp(g, 3, 6)
zx.simplify.spider_simp(g, quiet=True)
zx.simplify.id_simp(g, quiet=True)

# pauli web calculation
in_circ, out_circ = preprocess(g)
order, webs = compute_pauli_webs(g)
print(f'{in_circ.gates} {out_circ.gates}')

# highlight the web associated to the T spider
zx.draw(g, labels=True, pauli_web=webs[15])

[] []
